# Garbage Classification Project 

In [ ]:
!pip install tensorflow matplotlib

## Upload the Dataset ZIP file

In [ ]:
from google.colab import files
uploaded = files.upload()

## Extract the ZIP File

In [ ]:
import zipfile

with zipfile.ZipFile("Garbage_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("Garbage_dataset")

print("✅ Dataset Extracted!")

## Import Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

## Prepare the Data

In [ ]:
train_path = "Garbage_dataset/Garbage classification"
image_size = (150, 150)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


## Build the CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Train the Model

In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

## Plot Accuracy Graph

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()

## Predict a Custom Image

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=image_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_names = list(train_generator.class_indices.keys())
    predicted_class = class_names[np.argmax(prediction)]
    print("📌 Predicted Class:", predicted_class)

# predict_image("your_image.jpg")

## Save and Download the Model

In [ ]:
model.save("garbage_classifier_model.h5")
from google.colab import files
files.download("garbage_classifier_model.h5")